## Load Document $\rightarrow$ Split Document $\rightarrow$ **Storage** $\rightarrow$ retrieval $\rightarrow$ Storage Output

![Steps](image.jpg)

In [49]:

from pprint import pprint
import os
from dotenv import load_dotenv
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

### Emnbedding

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [3]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [4]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)


In [5]:
embedding1

[-0.02753432545375666,
 -0.005396349142994705,
 -0.0257534355955721,
 -0.033167495016985585,
 -0.027256456061340768,
 0.022482155867878245,
 -0.010369578899235891,
 -0.008216091108012741,
 0.002600288614640442,
 -0.019842396639927283,
 0.0006867793583776227,
 0.029201541808252,
 -0.00542792516798185,
 0.0006484933873353485,
 0.00031852351294530784,
 0.014183966815502165,
 0.030009888792980943,
 -0.0013317204487226831,
 0.004149094526188039,
 -0.004003844531850411,
 -0.011626305531914544,
 0.0070162007615174275,
 0.01304722898464241,
 -0.04695991986770518,
 -0.0023571528962765377,
 0.004742724634136648,
 0.016874247497168048,
 -0.00030530103210926036,
 -0.025791327384350197,
 -0.016192204612387687,
 0.02675123966154645,
 0.0029192069792380016,
 -0.015522792323866693,
 -0.024604068099775517,
 0.005920511437178072,
 -0.014967053539034911,
 0.009643328927547752,
 -0.011436849381991675,
 -0.004613262419461958,
 -0.010887425895289576,
 -0.016950029212079165,
 0.011064252380275618,
 0.0068204

In [6]:
import numpy as np

In [7]:
np.dot(embedding1, embedding2)

0.963182592636646

In [8]:
np.dot(embedding1, embedding3)

0.7710630976675918

In [9]:
np.dot(embedding2, embedding3)

0.7596504459474736

#### Load Document $\rightarrow$ Split Document $\rightarrow$ Store

In [10]:
from langchain.document_loaders import PyPDFLoader

In [11]:

loader = PyPDFLoader("docs/morin.pdf")
pages = loader.load()
len(pages)

11

In [12]:
pages[0].page_content

'GFK Meal Plan\n'

In [13]:
print(pages[3].page_content)

As always, tt’s recommended that you drink 8 glasses of water in a day .  FACT : W ater helps you lose weight and easy ways to maximize drinking water daily are:*When you wake up - 1 Glass*After Breakfast- 1 Glass*Midday- 1 Glass*Before Lunch- 1 Glass*Lunch- 1 Glass*After lunch- 1 Glass*Before Dinner- 1 Glass*Dinner- 1 GlassI can’t wait for your overall progress and I look for-ward to meeting YOUR ABS SOON!!!Good luck!April.3


##### Split Doc

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [15]:
splits = text_splitter.split_documents(pages)
splits

[Document(page_content='GFK Meal Plan', metadata={'source': 'docs/morin.pdf', 'page': 0}),
 Document(page_content='GFK Meal Plan\nCongratulations on your weight loss! From our last weigh-in, you’ve lost a total of 4.6kg and you are on  your healthy weight! Go Superwoman! So far, I’ll like to say WELL DONE and I’m very impressed with your commitment to the Lifestyle Change with April.No more excuses. W e’ve all got hurdles in life. Some are big and some are small but what’s important is that we ﬁnd a way past these challenges. Y ou’re on the right path. Y ou’re working out regularly and fueling your body with nutritious food. With this combo, nothing will stand in your way . Y ou can, and you will! Y our daily calorie goal is 1,250 per day and you should lose 2.2kg or more by 28/2/2017. Y our custom meal plan is ready and you’re one step closer to your goal weight!\n                                                                  Meal Plan for Maureen', metadata={'source': 'docs/morin.

In [16]:
len(splits) > 0

True

##### Vectorestores

In [17]:
# !pip install chromadb

In [18]:
from langchain.vectorstores import Chroma

In [19]:
persist_directory = 'docs/chroma/'

In [21]:
# !rm -rf ./docs/chroma  # remove old database files if any

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

In [23]:
print(vectordb._collection.count())

18


#### Similaryity text

In [24]:
question = " How many glasses of water should i drink"

In [25]:
docs = vectordb.similarity_search(question, k=3)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

In [26]:
len(docs)

3

In [27]:
print(docs[0].page_content)

As always, tt’s recommended that you drink 8 glasses of water in a day .  FACT : W ater helps you lose weight and easy ways to maximize drinking water daily are:*When you wake up - 1 Glass*After Breakfast- 1 Glass*Midday- 1 Glass*Before Lunch- 1 Glass*Lunch- 1 Glass*After lunch- 1 Glass*Before Dinner- 1 Glass*Dinner- 1 GlassI can’t wait for your overall progress and I look for-ward to meeting YOUR ABS SOON!!!Good luck!April.3


In [28]:
print(docs[1].page_content)

GFK Meal Plan
Congratulations on your weight loss! From our last weigh-in, you’ve lost a total of 4.6kg and you are on  your healthy weight! Go Superwoman! So far, I’ll like to say WELL DONE and I’m very impressed with your commitment to the Lifestyle Change with April.No more excuses. W e’ve all got hurdles in life. Some are big and some are small but what’s important is that we ﬁnd a way past these challenges. Y ou’re on the right path. Y ou’re working out regularly and fueling your body with nutritious food. With this combo, nothing will stand in your way . Y ou can, and you will! Y our daily calorie goal is 1,250 per day and you should lose 2.2kg or more by 28/2/2017. Y our custom meal plan is ready and you’re one step closer to your goal weight!
                                                                  Meal Plan for Maureen


In [29]:
print(docs[2].page_content)

GFK Meal Plan


In [30]:
question_2 = " what time should i drink water"

In [31]:
docs = vectordb.similarity_search(question_2, k=3)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

In [32]:
print(docs[0].page_content)

As always, tt’s recommended that you drink 8 glasses of water in a day .  FACT : W ater helps you lose weight and easy ways to maximize drinking water daily are:*When you wake up - 1 Glass*After Breakfast- 1 Glass*Midday- 1 Glass*Before Lunch- 1 Glass*Lunch- 1 Glass*After lunch- 1 Glass*Before Dinner- 1 Glass*Dinner- 1 GlassI can’t wait for your overall progress and I look for-ward to meeting YOUR ABS SOON!!!Good luck!April.3


In [33]:
print(docs[1].page_content)

GFK Meal Plan


### Limitation 1
Does not know about document structure

In [34]:
question_2 = " what time should i drink water"

In [35]:
docs = vectordb.similarity_search(question_2, k=3)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-UODwUjyYPzLPSUgqp1SkNJpG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

In [36]:
for doc in docs:
    print(doc.metadata)

{'page': 3, 'source': 'docs/morin.pdf'}
{'page': 0, 'source': 'docs/morin.pdf'}
{'page': 1, 'source': 'docs/morin.pdf'}


### Limitation 2
Does not give you unique codes

##### Loading

In [38]:
from langchain.document_loaders import PyPDFLoader

In [39]:
import glob

path = 'docs/css_lectures/'
files = glob.glob(path + '*.pdf')
files = sorted(files)
print(files)

['docs/css_lectures\\MachineLearning-Lecture01.pdf', 'docs/css_lectures\\MachineLearning-Lecture02.pdf', 'docs/css_lectures\\MachineLearning-Lecture03.pdf']


In [40]:
files.append(files[0])

In [42]:

files = sorted(files)
print(files)

['docs/css_lectures\\MachineLearning-Lecture01.pdf', 'docs/css_lectures\\MachineLearning-Lecture01.pdf', 'docs/css_lectures\\MachineLearning-Lecture02.pdf', 'docs/css_lectures\\MachineLearning-Lecture03.pdf']


In [39]:
# loaders = [PyPDFLoader("docs/morin.pdf"), PyPDFLoader("docs/morin.pdf")]


In [ ]:
# pages =[]
# for loader in loaders:
#     pages.extend(loader.load())
# len(pages)

In [43]:
pages = []
for file in files:
    loader = PyPDFLoader(file)
    pages.extend(loader.load())
len(pages)


78

##### splitting

In [44]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [45]:
splits = text_splitter.split_documents(pages)
len(splits)

209

In [48]:
splits[0].metadata

{'source': 'docs/css_lectures\\MachineLearning-Lecture01.pdf', 'page': 0}

##### vector storage

In [46]:
from langchain.vectorstores import Chroma

from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [31]:
persist_directory = 'docs/chroma/'

In [32]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

KeyboardInterrupt: 

In [33]:
vectordb._collection.count()

209

In [34]:
question = "what did they say about matlab?"

In [35]:
docs = vectordb.similarity_search(question,k=5)

In [36]:
pprint(docs[0].page_content)

('those homeworks will be done in either MATLA B or in Octave, which is sort '
 'of — I \n'
 'know some people call it a free ve rsion of MATLAB, which it sort  of is, '
 "sort of isn't.  \n"
 "So I guess for those of you that haven't s een MATLAB before, and I know "
 'most of you \n'
 'have, MATLAB is I guess part of the programming language that makes it very '
 'easy to write codes using matrices, to write code for numerical routines, to '
 'move data around, to \n'
 "plot data. And it's sort of an extremely easy to  learn tool to use for "
 'implementing a lot of \n'
 'learning algorithms.  \n'
 'And in case some of you want to work on your  own home computer or something '
 'if you \n'
 "don't have a MATLAB license, for the purposes of  this class, there's also — "
 '[inaudible] \n'
 "write that down [inaudible] MATLAB — there' s also a software package called "
 'Octave \n'
 'that you can download for free off the Internet. And it has somewhat fewer '
 "features than MATLAB, but

In [37]:
pprint(docs[1].page_content)

('those homeworks will be done in either MATLA B or in Octave, which is sort '
 'of — I \n'
 'know some people call it a free ve rsion of MATLAB, which it sort  of is, '
 "sort of isn't.  \n"
 "So I guess for those of you that haven't s een MATLAB before, and I know "
 'most of you \n'
 'have, MATLAB is I guess part of the programming language that makes it very '
 'easy to write codes using matrices, to write code for numerical routines, to '
 'move data around, to \n'
 "plot data. And it's sort of an extremely easy to  learn tool to use for "
 'implementing a lot of \n'
 'learning algorithms.  \n'
 'And in case some of you want to work on your  own home computer or something '
 'if you \n'
 "don't have a MATLAB license, for the purposes of  this class, there's also — "
 '[inaudible] \n'
 "write that down [inaudible] MATLAB — there' s also a software package called "
 'Octave \n'
 'that you can download for free off the Internet. And it has somewhat fewer '
 "features than MATLAB, but